In [1]:
from dataverk import Client, Datapackage
import dataverk_tools.themes.nav as nav

import pandas as pd
import ipywidgets as widgets
import datetime

import plotly.graph_objs as go
import plotly.io as pio
import plotly.express as px

pio.templates.default = nav.plotly_template

# Get data

In [2]:
url="https://www.nav.no/no/NAV+og+samfunn/Statistikk/Flere+statistikkomrader/Trygdemisbruk/_attachment/536372?_download=true&_ts=16140e9fd20"
df = pd.read_excel(url, skiprows=7)[1:20]
df.columns = ["Fylke", "Antall personer", "Anmeldt beløp"]

In [3]:
df.head(5)

,Fylke,Antall personer,Anmeldt beløp
1,Østfold,69.0,10639774.0
2,Akershus,108.0,22599182.0
3,Oslo,167.0,30772319.0
4,Hedmark,32.0,4669307.0
5,Oppland,21.0,3112811.0


In [4]:
url="https://www.nav.no/no/NAV+og+samfunn/Statistikk/Flere+statistikkomrader/Trygdemisbruk/_attachment/545663?_download=true&_ts=16894450da0"
df_2018 = pd.read_excel(url, skiprows=7)[1:20]
df_2018.columns = ["Fylke", "Antall personer", "Anmeldt beløp"]

In [5]:
df_2018.head()

,Fylke,Antall personer,Anmeldt beløp
1,Østfold,94.0,16917197.0
2,Akershus,99.0,16868268.0
3,Oslo,137.0,28588253.0
4,Hedmark,25.0,3802184.0
5,Oppland,17.0,2576129.0


# Create datapackage

In [20]:
readme = """
# Trygdemisbruk
"""

now = datetime.datetime.now().isoformat()
distribution_url = 'http://www.example.org'


metadata = {
'title': 'Trygdemisbruk 2017 - ',
'description': 'Trygdemisbruk 3x',
'readme': readme,
'accessRights': 'Open', # Optional. Default: Internal
'issued': now,
'modified': now, # Optional. Default: now
'language': 'Norsk', # Optional. Default: Norsk
'periodicity': 'Årlig', # Oppdateringsfrekvens
'temporal': {'from': '2017-01-01', 'to': '2017-12-31' },
'author': 'statistikk@nav.no',
#'publisher': {'name': 'Arbeids- og velferdsetaten (NAV)', 'url': 'https://www.nav.no'}, # Optional. Default NAV
#'contactpoint': {'name': 'NAV Kunnskapsavdelingen', 'email': 'statistikk@nav.no'},  
#'license': {'name': 'CC BY 4.0', 'url': 'http://creativecommons.org/licenses/by/4.0/deed.no'}, #Optional. Default CC BY 4.0
'keywords': ['trygdemisbruk'],
'spatial': 'Norge',
'theme': ['Åpne data'],
'type': 'datapackage',
'format': 'datapackage',
'provenance': 'Datavarehuset',
'repo': 'https://github.com/datasett/examples/blob/master/nav/trygdemisbruk.ipynb',
'notebook': 'https://github.com/datasett/examples/blob/master/nav/trygdemisbruk.ipynb',
'store': 'gs',
'project': 'odata', 
'bucket': 'nav-opendata'
}

In [21]:
dp = Datapackage(metadata)

# Create views

In [22]:
data = [ 
go.Bar(
    x=df['Fylke'], 
    y=df['Antall personer'],
    name='Antall personer'
),
go.Bar(
    x=df['Fylke'], 
    y=df['Anmeldt beløp'],
    name='Anmeldt beløp',
    yaxis='y2'
)
]

layout = go.Layout(
    yaxis=dict(
        title='Antall personer',
        side='left'
    ),
    yaxis2=dict(
        title='Anmeldt beløp',
        anchor='x',
        overlaying='y',
        side='right'
    ),
    barmode='group',
    template=nav.plotly_template
)

fig = go.Figure(data,layout)

go.FigureWidget(data,layout)

FigureWidget({
    'data': [{'name': 'Antall personer',
              'type': 'bar',
              'uid': 'ed3…

In [23]:
description = '''
Trygdemisbruk 2017 

'''

dp.add_view(
    spec_type='plotly',
    name='Trygdemisbruk 2017',
    title='Trygdemisbruk 2017',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=pio.to_json(fig)
)

In [24]:
df.dtypes

Fylke               object
Antall personer    float64
Anmeldt beløp      float64
dtype: object

In [25]:
df.head()

,Fylke,Antall personer,Anmeldt beløp
1,Østfold,69.0,10639774.0
2,Akershus,108.0,22599182.0
3,Oslo,167.0,30772319.0
4,Hedmark,32.0,4669307.0
5,Oppland,21.0,3112811.0


In [26]:
df.dtypes

Fylke               object
Antall personer    float64
Anmeldt beløp      float64
dtype: object

In [27]:
spec={'hidden': False, 
      'fields':[{'name': 'Fylke', 'description': 'Fylke'},
                {'name': 'Antall personer', 'description': 'Antall personer'},
                {'name': 'Anmeldt beløp', 'description': 'Anmeldt beløp i hele kroner'}]}

In [28]:
dp.add_resource(df, 'Trygdemisbruk_2017', dsv_separator=';', spec=spec)

In [29]:
dp.add_view(
    spec_type='table',
    name='Trygdemisbruk 2017',
    resources='Trygdemisbruk_2017',
)

In [30]:
dp.add_resource(df_2018, 'Trygdemisbruk_2018', 'Trygdemisbruk 2018. Kilde: NAV', spec=spec)

dp.add_view(
    spec_type='table',
    name='Trygdemisbruk 2018',
    title='2018 tabell',
    description = 'Trygdemisbruk 2018',
    resources='Trygdemisbruk_2018',
)

In [31]:
spec_hidden = {'hidden': True, 
      'fields':[{'name': 'Fylke', 'description': 'Fylke'},
                {'name': 'Antall personer', 'description': 'Antall personer'},
                {'name': 'Anmeldt beløp', 'description': 'Anmeldt beløp i hele kroner'}]}

dp.add_resource(df_2018, 'Trygdemisbruk_2019', spec=spec_hidden)

dp.add_view(
    spec_type='table',
    name='Trygdemisbruk 2019',
    title='2019 tabell',
    description = 'Trygdemisbruk 2019',
    resources='Trygdemisbruk_2019',
)

# Save data and publish datapackage

In [32]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/users/b149899/projects/binder/binder/keys/nav_opendata.json'


In [33]:
client = Client()
client.publish(dp)

gs
2019-11-01T00:00:53.719159: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: json) written to https://storage.googleapis.com/nav-opendata/665f912b6789d9f300ef28d8706b0969/datapackage.json
2019-11-01T00:00:54.029005: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: csv.gz) written to https://storage.googleapis.com/nav-opendata/665f912b6789d9f300ef28d8706b0969/resources/Trygdemisbruk_2017.csv.gz
2019-11-01T00:00:54.328372: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: csv.gz) written to https://storage.googleapis.com/nav-opendata/665f912b6789d9f300ef28d8706b0969/resources/Trygdemisbruk_2018.csv.gz
2019-11-01T00:00:54.586514: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: csv.gz) written to https://storage.googleapis.com/nav-opendata/665f912b6789d9f300ef28d8706b0969/resources/Trygdemisbruk_2019.csv.gz
2019-1